<a href="https://colab.research.google.com/github/eduardojdiniz/CichyWanderers/blob/b49ac01ec04b7d391eef75631c709dfd8661830c/RDMs_loader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RDMs loader

In [ ]:
%load_ext autoreload
%autoreload 2

### Import CichyWanderers GitHub Repository

In [ ]:
!git clone https://github.com/eduardojdiniz/CichyWanderers

### Import Data loader

In [ ]:
import CichyWanderers.dataloader as dataloader

### Import AlexNet loader

In [ ]:
import CichyWanderers.alexnet as alexnet 

### Download and create Cichy et al, 2014 dataset

In [ ]:
_, RDM_dict, stim_dict = dataloader.create_dataset()

### Download and create AlexNet model (pretrained on ImageNet) RDMs

In [ ]:
alexnet_RDM_dict = alexnet.create_alexnet_RDM_dict()

<details>
    
<summary><font color='blue'>Click here for details about the dictionaries structure</font>
</summary>

### Dictionaries structures
    
#### `RDM_dict`
    RDM_dict: dict with keys 'MEG', 'fMRI_EVC', 'fMRI_IT'
    'MEG'     : ndarray, (16, 2, 1301, 92, 92)
        16 subjects, 2 sessions, 1301 time points (from -100 ms to 1200 ms
        wrt to stimulus onset at 0 ms), 92 conditions by 92 conditions.
        The last 2 dimensions form representational dissimilarity matrices of
        decoding accuracies, symmetric accross the diagonal, with the diagonal
        undefined (NaN).
    'fMRI_EVC': ndarray, (15, 92, 92)
        15 subjects, 92 conditions by 92 conditions.
        The last 2 dimensions form a representational dissimilarity matrix of
        spearman correlation for the EVC cortex, symmetric accross the diagonal,
        with the diagonal undefined (NaN).
    'fMRI_IT' : ndarray, (15, 92, 92)
        15 subjects, 92 conditions by 92 conditions.
        The last 2 dimensions form a representational dissimilarity matrix of
        spearman correlation for the IT cortex, symmetric accross the diagonal,
        with the diagonal undefined (NaN).
        
#### `stim_dict`        
    stim_dict: dict with keys 'category', 'human', 'face', 'animate', 'natural', 'imagepath'
    'category'  : list[str], indicating category
    'human'     : list[int], indicating membership (0=not a member, 1=member)
    'face'      : list[int], indicating membership (0=not a member, 1=member)
    'animate'   : list[int], indicating membership (0=not a member, 1=member)
    'natural'   : list[int], indicating membership (0=not a member, 1=member)
    'imagepath' : list[str], jpeg image filepath

#### `alexnet_RDM_dict`
    alexnet_RDM_dict: dict with keys 'layer_1', ..., 'layer_8'
    Each item is a ndarray, (92, 92), a representational dissimilarity matrix of
    spearman correlation of the layer activation to each pair of visual stimuli
    
</details>

## Partialling Out the Representational Dissimilarities based on Low Level Image Statistics 

Different filters in different layers of Deep Convolutional Neural Networks such as AlexNet are trying to highlight or activate different parts of the image. Some filters are acting as edge, texture, and color detectors, others are detecting a particular region and still others are acting as background detectors. 

In [ ]:
# import
import numpy as np
from sklearn.linear_model import LinearRegression

In [ ]:
# Average RDMs across subjects and sessions
MEG_RDMs = np.mean(RDM_dict['MEG'], axis=(0,1))
fMRI_EVC = np.mean(RDM_dict['fMRI_EVC'], axis=0)
fMRI_IT = np.mean(RDM_dict['fMRI_IT'], axis=0)
layer_1 = alexnet_RDM_dict['layer_1'] 
layer_2 = alexnet_RDM_dict['layer_2'] 

In [ ]:
def rescale(x):
    return ((x - x.min()) * (1/(x.max() - x.min()))).astype('float32')

def get_residual_RDM(X, y):
  
  # Indices of lower triangular matrix, excluding the diagonal
  n_stim = X.shape[0]
  li = np.tril_indices(n_stim, k=-1) 
  di = np.diag_indices(n_stim)
    
  X_flat = X[li].reshape((-1,1))
  y_flat = y[li].reshape((-1,1)) 
  #X_rescaled = rescale(X_flat)
  #y_rescaled = rescale(y_flat)
  X_rescaled = X_flat 
  y_rescaled = y_flat
  reg = LinearRegression().fit(X_rescaled, y_rescaled)
  res = y_rescaled - reg.predict(X_rescaled)
  # Fill low triangular matrix with residuals
  RDM = np.zeros_like(X)
  RDM[li] = res.squeeze()
   
  # Make it symmetrical
  RDM = np.tril(RDM) + np.triu(RDM.T, 1)

  # Restore original diagonal  
  #RDM[di] = X[di]

  return RDM

In [ ]:
# Total duration of stimulus presentation, in ms
T_stim = MEG_RDMs.shape[0]

# Number of stimuli
n_stim = MEG_RDMs.shape[1]

resRDM_dict = {'layer_1': {}, 'layer_2': {}}

# Get the residual RDMs for MEG
resMEG_RDMs_layer1 = np.zeros_like(MEG_RDMs)
resMEG_RDMs_layer2 = np.zeros_like(MEG_RDMs)
for t in range(0, T_stim):
  # Load MEG RDM at a given timepoint 
  # +100 as the RDMs provided are from -100ms to 1300ms after the stimulus onset
  RDM = MEG_RDMs[t]
  resMEG_RDMs_layer1[t] = get_residual_RDM(layer_1, RDM)
  resMEG_RDMs_layer2[t] = get_residual_RDM(layer_2, RDM)
resRDM_dict['layer_1']['MEG'] = resMEG_RDMs_layer1
resRDM_dict['layer_2']['MEG'] = resMEG_RDMs_layer2
del RDM, resMEG_RDMs_layer1, resMEG_RDMs_layer2

# Get the residual RDMs for fMRI
resRDM_dict['layer_1']['fMRI_EVC'] = get_residual_RDM(layer_1, fMRI_EVC)
resRDM_dict['layer_2']['fMRI_EVC'] = get_residual_RDM(layer_2, fMRI_EVC)
resRDM_dict['layer_1']['fMRI_IT'] = get_residual_RDM(layer_1, fMRI_IT)
resRDM_dict['layer_2']['fMRI_IT'] = get_residual_RDM(layer_2, fMRI_IT)

In [ ]:
from RDM_plot import plot_rdm

In [ ]:
from ipywidgets import widgets
@widgets.interact( resRDMs=widgets.fixed(resRDM_dict["layer_1"]["MEG"]),
                   RDMs=widgets.fixed(MEG_RDMs),
                   percentile=widgets.Checkbox(value=True, description="percentile"),
                   timepoint=widgets.IntSlider(min=0, max=1200, step=10, value=500, description='t (ms):') )
def plot_RDMs(resRDMs, RDMs,timepoint=0, percentile=False):
    """Helper function for visualize MEG RDMs with an interactive 
    slider for the timepoint."""
    # Load RDM at a given timepoint 
    # +100 as the RDMs provided are from -100ms to 1000ms after the stimulus onset
    resRDM = np.array(resRDMs[timepoint+100])
    RDM = np.array(RDMs[timepoint+100])
    title_res = "Residual MEG RDM at t = " + str(timepoint) + " ms"
    title = "MEG RDM at t = " + str(timepoint) + " ms"
    plot_rdm(resRDM, percentile=percentile, title=title_res)
    plot_rdm(RDM, percentile=percentile, title=title)

In [ ]:
# Plot fMRI EVC residual RDM
title_res = "Residual fMRI EVC RDM"
title = "fMRI EVC RDM"
plot_rdm(resRDM_dict["layer_1"]["fMRI_EVC"], percentile=True, title=title_res)
plot_rdm(layer_1, percentile=True, title=title)

In [ ]:
# Plot fMRI EVC residual RDM
title_res = "Residual fMRI IT RDM"
title = "fMRI IT RDM"
plot_rdm(resRDM_dict["layer_1"]["fMRI_IT"], percentile=True, title=title_res)
plot_rdm(fMRI_IT, percentile=True, title=title)

# MEG-fMRI comparison: To find out at which timepoint MEG representation is similar to a given ROI's representation. 

In [ ]:
# RDM Comparison functions

from scipy.stats import spearmanr

def RSA_spearman(rdm1,rdm2):
    """
    computes and returns the spearman correlation between lower triangular 
    part of the input rdms. We only need to compare either lower or upper 
    triangular part of the matrix as RDM is symmetric
    """
    # get lower triangular part of the RDM1 
    lt_rdm1 = get_lowertriangular(rdm1)
    # get lower triangular part of the RDM1 
    lt_rdm2 = get_lowertriangular(rdm2)
    # return Spearman's correlation between lower triangular part of rdm1 & rdm2
    return spearmanr(lt_rdm1, lt_rdm2)[0]

def get_lowertriangular(rdm):
    """
    returns lower triangular part of the matrix
    """
    num_conditions = rdm.shape[0]
    return rdm[np.tril_indices(num_conditions,-1)]

In [ ]:
# Correlating MEG RDMs with fMRI RDMs
num_timepoints =resRDM_dict["layer_1"]["MEG"].shape[0] # get number of timepoints

# initialize a dictionary to store MEG and ROI RDM correlation at each timepoint
MEG_correlation = {}
ROIs = ['EVC_1','IT_1','EVC_2','IT_2', 'EVC', 'IT']
for ROI in ROIs:
  MEG_correlation[ROI] = []

# for loop that goes over MEG RDMs at all time points and correlate with ROI RDMs
for t in range(num_timepoints):
  MEG_RDM_t = resRDM_dict["layer_1"]["MEG"][t,:,:]
  MEG_correlation['EVC_1'].append(RSA_spearman(fMRI_EVC, MEG_RDM_t))
  MEG_correlation['IT_1'].append(RSA_spearman(fMRI_IT, MEG_RDM_t))
  #MEG_correlation['EVC_1'].append(RSA_spearman(resRDM_dict["layer_1"]['fMRI_EVC'], MEG_RDM_t))
  #MEG_correlation['IT_1'].append(RSA_spearman(resRDM_dict["layer_1"]['fMRI_IT'], MEG_RDM_t))
  MEG_RDM_t = resRDM_dict["layer_2"]["MEG"][t,:,:]
  MEG_correlation['EVC_2'].append(RSA_spearman(fMRI_EVC, MEG_RDM_t))
  MEG_correlation['IT_2'].append(RSA_spearman(fMRI_IT, MEG_RDM_t))
  #MEG_correlation['EVC_2'].append(RSA_spearman(resRDM_dict["layer_2"]['fMRI_EVC'], MEG_RDM_t))
  #MEG_correlation['IT_2'].append(RSA_spearman(resRDM_dict["layer_2"]['fMRI_IT'], MEG_RDM_t))
  MEG_RDM_t = MEG_RDMs[t,:,:]
  MEG_correlation['EVC'].append(RSA_spearman(fMRI_EVC, MEG_RDM_t))
  MEG_correlation['IT'].append(RSA_spearman(fMRI_IT, MEG_RDM_t))
  #for ROI in ROIs:
  #  ROI_RDM = resRDM_dict["layer_2"]['fMRI_'+ ROI]
  #  MEG_correlation[ROI].append(RSA_spearman(ROI_RDM, MEG_RDM_t))

In [ ]:
# Plotting MEG-fMRI comparison
import matplotlib.pyplot as plt
# for Palatino and other serif fonts use:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})
plt.rc('font', size=12)
fig, ax = plt.subplots(figsize=(10, 6))

time_range = range(-100,1201)
ax.plot(time_range, MEG_correlation['IT'], color='tab:brown', label=r'$\rho(\textrm{IT}, \textrm{MEG}(t))$')
ax.plot(time_range, MEG_correlation['IT_1'], color='tab:orange', label=r'$\rho(\textrm{IT}, \textrm{MEG}(t)-\textrm{Layer 1})$')

ax.plot(time_range, MEG_correlation['IT_2'], color='tab:red', label=r'$\rho(\textrm{IT}, \textrm{MEG}(t)-\textrm{Layer 2})$')
ax.plot(time_range, MEG_correlation['EVC'], color='tab:gray', label=r'$\rho(\textrm{EVC}, \textrm{MEG}(t))$')
ax.plot(time_range, MEG_correlation['EVC_1'], color='tab:blue', label=r'$\rho(\textrm{EVC}, \textrm{MEG}(t)-\textrm{Layer 1})$')
ax.plot(time_range, MEG_correlation['EVC_2'], color='tab:purple', label=r'$\rho(\textrm{EVC}, \textrm{MEG}(t)-\textrm{Layer 2})$')

# Same as above
ax.set_xlabel('Time')
ax.set_ylabel('Spearmans Correlation')
ax.set_title('MEG-fMRI fusion')
ax.grid(True)
ax.legend(loc='upper right');

# Permutation Test

In [ ]:
from RSA import RSA

In [ ]:
a,b = np.shape(fMRI_IT)
similarities = RSA(fMRI_IT, MEG_RDMs[100:400], permutation=True, n_iter=10)